In [57]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from xgboost import plot_importance
import xgboost as xgb

import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None)

In [58]:
df = pd.read_csv('./../data/Father.csv')
df

,#CHROM,ID,REF,ALT,Consequence annotations from Ensembl VEP. Format: Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,REFSEQ_MATCH,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,ada_score,rf_score,BLOSUM62,CADD_phred,CADD_raw,CADD_raw_rankscore,DANN_rankscore,DANN_score,Eigen-PC-raw,Eigen-raw,FATHMM_pred,GERP++_NR,GERP++_RS,LRT_pred,M-CAP_pred,MetaLR_pred,MetaSVM_pred,MutPred_rankscore,MutPred_score,MutationAssessor_pred,MutationTaster_pred,PROVEAN_pred,Polyphen2_HDIV_pred,Polyphen2_HVAR_pred,REVEL_rankscore,REVEL_score,SIFT_pred,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,VEST3_rankscore,VEST3_score,phastCons100way_vertebrate,phastCons20way_mammalian,phyloP100way_vertebrate,phyloP20way_mammalian,ClinVar,ClinVar_ALLELEID,ClinVar_CLNDN,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,HGMD,HGMD_CLASS,HGMD_PHENO,HGMD_PUBMED,HGMD_RANKSCORE,DOMAINS_Pfam,SuperDups,gnomadWES,gnomadWES_AC_clean,gnomadWES_AF_clean,gnomadWES_AN_clean,gnomadWES_AC_Male,gnomadWES_AC_Female,gnomadWES_AN_Male,gnomadWES_AN_Female,gnomadWES_AF_Male,gnomadWES_AF_Female,gnomadWES_AC_raw,gnomadWES_AN_raw,gnomadWES_AF_raw,gnomadWES_Hom_AFR,gnomadWES_Hom_AMR,gnomadWES_Hom_ASJ,gnomadWES_Hom_EAS,gnomadWES_Hom_FIN,gnomadWES_Hom_NFE,gnomadWES_Hom_OTH,gnomadWES_Hom_clean,gnomadWES_Hom_raw,gnomadWES_AC_AFR,gnomadWES_AC_AMR,gnomadWES_AC_ASJ,gnomadWES_AC_EAS,gnomadWES_AC_FIN,gnomadWES_AC_NFE,gnomadWES_AC_OTH,gnomadWES_AN_AFR,gnomadWES_AN_AMR,gnomadWES_AN_ASJ,gnomadWES_AN_EAS,gnomadWES_AN_FIN,gnomadWES_AN_NFE,gnomadWES_AN_OTH,gnomadWES_AF_AFR,gnomadWES_AF_AMR,gnomadWES_AF_ASJ,gnomadWES_AF_EAS,gnomadWES_AF_SAS,gnomadWES_AF_FIN,gnomadWES_AF_NFE,gnomadWES_AF_OTH,gnomadWES_POPMAX,gnomadWES_AC_POPMAX,gnomadWES_AN_POPMAX,gnomadWES_AF_POPMAX,gnomadWES_Hom_Male,gnomadWES_Hom_Female,gnomadWGS,gnomadWGS_AC_clean,gnomadWGS_AF_clean,gnomadWGS_AN_clean,gnomadWGS_AC_Male,gnomadWGS_AC_Female,gnomadWGS_AN_Male,gnomadWGS_AN_Female,gnomadWGS_AF_Male,gnomadWGS_AF_Female,gnomadWGS_AC_raw,gnomadWGS_AN_raw,gnomadWGS_AF_raw,gnomadWGS_Hom_AFR,gnomadWGS_Hom_AMR,gnomadWGS_Hom_ASJ,gnomadWGS_Hom_EAS,gnomadWGS_Hom_FIN,gnomadWGS_Hom_NFE,gnomadWGS_Hom_OTH,gnomadWGS_Hom_clean,gnomadWGS_Hom_raw,gnomadWGS_AC_AFR,gnomadWGS_AC_AMR,gnomadWGS_AC_ASJ,gnomadWGS_AC_EAS,gnomadWGS_AC_FIN,gnomadWGS_AC_NFE,gnomadWGS_AC_OTH,gnomadWGS_AN_AFR,gnomadWGS_AN_AMR,gnomadWGS_AN_ASJ,gnomadWGS_AN_EAS,gnomadWGS_AN_FIN,gnomadWGS_AN_NFE,gnomadWGS_AN_OTH,gnomadWGS_AF_AFR,gnomadWGS_AF_AMR,gnomadWGS_AF_ASJ,gnomadWGS_AF_EAS,gnomadWGS_AF_FIN,gnomadWGS_AF_NFE,gnomadWGS_AF_OTH,gnomadWGS_POPMAX,gnomadWGS_AC_POPMAX,gnomadWGS_AN_POPMAX,gnomadWGS_AF_POPMAX,gnomadWGS_Hom_Male,gnomadWGS_Hom_Female,Cosmic,Cosmic_LEGACY_ID,Cosmic_CNT,Mastermind,Mastermind_MMURI3
0,chr1,rs75062661,A,G,G,missense_variant,MODERATE,OR4F5,79501.0,Transcript,NM_001005484.1,protein_coding,1/1,NaN,NM_001005484.1:c.421A>G,NP_001005484.1:p.Thr141Ala,421,421,141,T/A,Aca/Gca,rs2691305,NaN,1.0,NaN,SNV,EntrezGene,NaN,YES,NaN,NaN,NaN,NP_001005484.1,NaN,NaN,NaN,NaN,NaN,NaN,tolerated(0.82),benign(0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5441,0.88740,0.949700,0.6075,0.9514,0.9767,0.9994,0.9916,0.972600,0.9506,0.9854,0.99940,gnomAD_EAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047,-0.783573,0.04092,0.03511,0.453539,-1.340569,-1.450103,T,2.31,1.15,N,NaN,T,T,NaN,NaN,N,P,N,B,B,0.15395,0.053,T,4.1978,0.09838,0.0013,0.012,0.000,0.765,-0.684,-0.132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7tm_4&7TM_GPCR_Srsx&7tm_1,NaN,rs2691305,147701.0,0.950616,155374.0,82012.0,65689.0,85560.0,6

In [59]:
con_split = df['Consequence'].apply(lambda x: x if pd.isna(x) else x.split('&'))
mlb = MultiLabelBinarizer()
con_df = pd.DataFrame(mlb.fit_transform(con_split), columns=mlb.classes_)

df_copy = df.merge(con_df, left_on=df.index, right_on=con_df.index)
df_copy.drop('key_0', axis=1, inplace=True)
df_copy

,#CHROM,ID,REF,ALT,Consequence annotations from Ensembl VEP. Format: Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,REFSEQ_MATCH,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,ada_score,rf_score,BLOSUM62,CADD_phred,CADD_raw,CADD_raw_rankscore,DANN_rankscore,DANN_score,Eigen-PC-raw,Eigen-raw,FATHMM_pred,GERP++_NR,GERP++_RS,LRT_pred,M-CAP_pred,MetaLR_pred,MetaSVM_pred,MutPred_rankscore,MutPred_score,MutationAssessor_pred,MutationTaster_pred,PROVEAN_pred,Polyphen2_HDIV_pred,Polyphen2_HVAR_pred,REVEL_rankscore,REVEL_score,SIFT_pred,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,VEST3_rankscore,VEST3_score,phastCons100way_vertebrate,phastCons20way_mammalian,phyloP100way_vertebrate,phyloP20way_mammalian,ClinVar,ClinVar_ALLELEID,ClinVar_CLNDN,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,HGMD,HGMD_CLASS,HGMD_PHENO,HGMD_PUBMED,HGMD_RANKSCORE,DOMAINS_Pfam,SuperDups,gnomadWES,gnomadWES_AC_clean,gnomadWES_AF_clean,gnomadWES_AN_clean,gnomadWES_AC_Male,gnomadWES_AC_Female,gnomadWES_AN_Male,gnomadWES_AN_Female,gnomadWES_AF_Male,gnomadWES_AF_Female,gnomadWES_AC_raw,gnomadWES_AN_raw,gnomadWES_AF_raw,gnomadWES_Hom_AFR,gnomadWES_Hom_AMR,gnomadWES_Hom_ASJ,gnomadWES_Hom_EAS,gnomadWES_Hom_FIN,gnomadWES_Hom_NFE,gnomadWES_Hom_OTH,gnomadWES_Hom_clean,gnomadWES_Hom_raw,gnomadWES_AC_AFR,gnomadWES_AC_AMR,gnomadWES_AC_ASJ,gnomadWES_AC_EAS,gnomadWES_AC_FIN,gnomadWES_AC_NFE,gnomadWES_AC_OTH,gnomadWES_AN_AFR,gnomadWES_AN_AMR,gnomadWES_AN_ASJ,gnomadWES_AN_EAS,gnomadWES_AN_FIN,gnomadWES_AN_NFE,gnomadWES_AN_OTH,gnomadWES_AF_AFR,gnomadWES_AF_AMR,gnomadWES_AF_ASJ,gnomadWES_AF_EAS,gnomadWES_AF_SAS,gnomadWES_AF_FIN,gnomadWES_AF_NFE,gnomadWES_AF_OTH,gnomadWES_POPMAX,gnomadWES_AC_POPMAX,gnomadWES_AN_POPMAX,gnomadWES_AF_POPMAX,gnomadWES_Hom_Male,gnomadWES_Hom_Female,gnomadWGS,gnomadWGS_AC_clean,gnomadWGS_AF_clean,gnomadWGS_AN_clean,gnomadWGS_AC_Male,gnomadWGS_AC_Female,gnomadWGS_AN_Male,gnomadWGS_AN_Female,gnomadWGS_AF_Male,gnomadWGS_AF_Female,gnomadWGS_AC_raw,gnomadWGS_AN_raw,gnomadWGS_AF_raw,gnomadWGS_Hom_AFR,gnomadWGS_Hom_AMR,gnomadWGS_Hom_ASJ,gnomadWGS_Hom_EAS,gnomadWGS_Hom_FIN,gnomadWGS_Hom_NFE,gnomadWGS_Hom_OTH,gnomadWGS_Hom_clean,gnomadWGS_Hom_raw,gnomadWGS_AC_AFR,gnomadWGS_AC_AMR,gnomadWGS_AC_ASJ,gnomadWGS_AC_EAS,gnomadWGS_AC_FIN,gnomadWGS_AC_NFE,gnomadWGS_AC_OTH,gnomadWGS_AN_AFR,gnomadWGS_AN_AMR,gnomadWGS_AN_ASJ,gnomadWGS_AN_EAS,gnomadWGS_AN_FIN,gnomadWGS_AN_NFE,gnomadWGS_AN_OTH,gnomadWGS_AF_AFR,gnomadWGS_AF_AMR,gnomadWGS_AF_ASJ,gnomadWGS_AF_EAS,gnomadWGS_AF_FIN,gnomadWGS_AF_NFE,gnomadWGS_AF_OTH,gnomadWGS_POPMAX,gnomadWGS_AC_POPMAX,gnomadWGS_AN_POPMAX,gnomadWGS_AF_POPMAX,gnomadWGS_Hom_Male,gnomadWGS_Hom_Female,Cosmic,Cosmic_LEGACY_ID,Cosmic_CNT,Mastermind,Mastermind_MMURI3,3_prime_UTR_variant,5_prime_UTR_variant,coding_sequence_variant,downstream_gene_variant,frameshift_variant,incomplete_terminal_codon_variant,inframe_deletion,inframe_insertion,intergenic_variant,intron_variant,missense_variant,non_coding_transcript_exon_variant,non_coding_transcript_variant,protein_altering_variant,splice_acceptor_variant,splice_donor_variant,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant
0,chr1,rs75062661,A,G,G,missense_variant,MODERATE,OR4F5,79501.0,Transcript,NM_001005484.1,protein_coding,1/1,NaN,NM_001005484.1:c.421A>G,NP_001005484.1:p.Thr141Ala,421,421,141,T/A,Aca/Gca,rs2691305,NaN,1.0,NaN,SNV,EntrezGene,NaN,YES,NaN,NaN,NaN,NP_001005484.1,NaN,NaN,NaN,NaN,NaN,NaN,tolerat

In [60]:
df_IN = df_copy[df_copy['INTRON'].notna()]
df_IN

,#CHROM,ID,REF,ALT,Consequence annotations from Ensembl VEP. Format: Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,REFSEQ_MATCH,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,ada_score,rf_score,BLOSUM62,CADD_phred,CADD_raw,CADD_raw_rankscore,DANN_rankscore,DANN_score,Eigen-PC-raw,Eigen-raw,FATHMM_pred,GERP++_NR,GERP++_RS,LRT_pred,M-CAP_pred,MetaLR_pred,MetaSVM_pred,MutPred_rankscore,MutPred_score,MutationAssessor_pred,MutationTaster_pred,PROVEAN_pred,Polyphen2_HDIV_pred,Polyphen2_HVAR_pred,REVEL_rankscore,REVEL_score,SIFT_pred,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,VEST3_rankscore,VEST3_score,phastCons100way_vertebrate,phastCons20way_mammalian,phyloP100way_vertebrate,phyloP20way_mammalian,ClinVar,ClinVar_ALLELEID,ClinVar_CLNDN,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,HGMD,HGMD_CLASS,HGMD_PHENO,HGMD_PUBMED,HGMD_RANKSCORE,DOMAINS_Pfam,SuperDups,gnomadWES,gnomadWES_AC_clean,gnomadWES_AF_clean,gnomadWES_AN_clean,gnomadWES_AC_Male,gnomadWES_AC_Female,gnomadWES_AN_Male,gnomadWES_AN_Female,gnomadWES_AF_Male,gnomadWES_AF_Female,gnomadWES_AC_raw,gnomadWES_AN_raw,gnomadWES_AF_raw,gnomadWES_Hom_AFR,gnomadWES_Hom_AMR,gnomadWES_Hom_ASJ,gnomadWES_Hom_EAS,gnomadWES_Hom_FIN,gnomadWES_Hom_NFE,gnomadWES_Hom_OTH,gnomadWES_Hom_clean,gnomadWES_Hom_raw,gnomadWES_AC_AFR,gnomadWES_AC_AMR,gnomadWES_AC_ASJ,gnomadWES_AC_EAS,gnomadWES_AC_FIN,gnomadWES_AC_NFE,gnomadWES_AC_OTH,gnomadWES_AN_AFR,gnomadWES_AN_AMR,gnomadWES_AN_ASJ,gnomadWES_AN_EAS,gnomadWES_AN_FIN,gnomadWES_AN_NFE,gnomadWES_AN_OTH,gnomadWES_AF_AFR,gnomadWES_AF_AMR,gnomadWES_AF_ASJ,gnomadWES_AF_EAS,gnomadWES_AF_SAS,gnomadWES_AF_FIN,gnomadWES_AF_NFE,gnomadWES_AF_OTH,gnomadWES_POPMAX,gnomadWES_AC_POPMAX,gnomadWES_AN_POPMAX,gnomadWES_AF_POPMAX,gnomadWES_Hom_Male,gnomadWES_Hom_Female,gnomadWGS,gnomadWGS_AC_clean,gnomadWGS_AF_clean,gnomadWGS_AN_clean,gnomadWGS_AC_Male,gnomadWGS_AC_Female,gnomadWGS_AN_Male,gnomadWGS_AN_Female,gnomadWGS_AF_Male,gnomadWGS_AF_Female,gnomadWGS_AC_raw,gnomadWGS_AN_raw,gnomadWGS_AF_raw,gnomadWGS_Hom_AFR,gnomadWGS_Hom_AMR,gnomadWGS_Hom_ASJ,gnomadWGS_Hom_EAS,gnomadWGS_Hom_FIN,gnomadWGS_Hom_NFE,gnomadWGS_Hom_OTH,gnomadWGS_Hom_clean,gnomadWGS_Hom_raw,gnomadWGS_AC_AFR,gnomadWGS_AC_AMR,gnomadWGS_AC_ASJ,gnomadWGS_AC_EAS,gnomadWGS_AC_FIN,gnomadWGS_AC_NFE,gnomadWGS_AC_OTH,gnomadWGS_AN_AFR,gnomadWGS_AN_AMR,gnomadWGS_AN_ASJ,gnomadWGS_AN_EAS,gnomadWGS_AN_FIN,gnomadWGS_AN_NFE,gnomadWGS_AN_OTH,gnomadWGS_AF_AFR,gnomadWGS_AF_AMR,gnomadWGS_AF_ASJ,gnomadWGS_AF_EAS,gnomadWGS_AF_FIN,gnomadWGS_AF_NFE,gnomadWGS_AF_OTH,gnomadWGS_POPMAX,gnomadWGS_AC_POPMAX,gnomadWGS_AN_POPMAX,gnomadWGS_AF_POPMAX,gnomadWGS_Hom_Male,gnomadWGS_Hom_Female,Cosmic,Cosmic_LEGACY_ID,Cosmic_CNT,Mastermind,Mastermind_MMURI3,3_prime_UTR_variant,5_prime_UTR_variant,coding_sequence_variant,downstream_gene_variant,frameshift_variant,incomplete_terminal_codon_variant,inframe_deletion,inframe_insertion,intergenic_variant,intron_variant,missense_variant,non_coding_transcript_exon_variant,non_coding_transcript_variant,protein_altering_variant,splice_acceptor_variant,splice_donor_variant,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant
2,chr1,rs4372192,A,G,G,intron_variant,MODIFIER,SAMD11,148398.0,Transcript,NM_152486.2,protein_coding,NaN,7/13,NM_152486.2:c.707-25A>G,NaN,NaN,NaN,NaN,NaN,NaN,rs4372192,NaN,1.0,NaN,SNV,EntrezGene,NaN,NaN,NaN,NaN,NaN,NP_689699.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9131,0.8737,0.9207

In [61]:
model_in = joblib.load('./bst_in_rf.pkl')

in_cols = ['IMPACT',
       'AF', 'ada_score', 'rf_score',
       '3_prime_UTR_variant', '5_prime_UTR_variant', 'coding_sequence_variant',
       'downstream_gene_variant', 'frameshift_variant', 'inframe_deletion',
       'inframe_insertion', 'intergenic_variant', 'intron_variant',
       'missense_variant', 'protein_altering_variant',
       'splice_acceptor_variant', 'splice_donor_variant',
       'splice_region_variant', 'start_lost', 'start_retained_variant',
       'stop_gained', 'stop_lost', 'stop_retained_variant',
       'synonymous_variant', 'upstream_gene_variant']
       # 'transcript_ablation'


In [62]:
# MAX_AF
df_IN['MAX_AF'].fillna(0)

# ada_score
df_IN['ada_score'] = df_IN['ada_score'].fillna(df_IN.groupby('Consequence')['ada_score'].transform('mean'))

# rf_score
df_IN['rf_score'] = df_IN['rf_score'].fillna(df_IN.groupby('Consequence')['rf_score'].transform('mean'))

#
df_IN['gnomadWES_AF_POPMAX'] = df_IN['gnomadWES_AF_POPMAX'].apply(lambda x: 0 if (pd.isna(x)) or (x == '.') else float(x))
df_IN['AF'] = df_IN.apply(lambda x: np.mean([x['MAX_AF'], float(x['gnomadWES_AF_POPMAX'])]), axis=1)


le = LabelEncoder()
# IMPACT
df_IN['IMPACT'] = le.fit_transform(df_IN['IMPACT'])


inputs_in = df_IN[in_cols]
inputs_in['transcript_ablation'] = 0


In [63]:
inputs_in

,IMPACT,ada_score,rf_score,3_prime_UTR_variant,5_prime_UTR_variant,coding_sequence_variant,downstream_gene_variant,frameshift_variant,inframe_deletion,inframe_insertion,intergenic_variant,intron_variant,missense_variant,protein_altering_variant,splice_acceptor_variant,splice_donor_variant,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant,transcript_ablation
2,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,0.000078,0.040000,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
13,1,0.000050,0.054000,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
14,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120172,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
120173,1,0.037854,0.056886,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
120174,1,0.037854,0.056886,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
120175,1,0.037854,0.056886,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [64]:
# res = model_in.predict(inputs_in)
df_IN['predict'] = model_in.predict(inputs_in)

In [65]:
inputs_in

,IMPACT,ada_score,rf_score,3_prime_UTR_variant,5_prime_UTR_variant,coding_sequence_variant,downstream_gene_variant,frameshift_variant,inframe_deletion,inframe_insertion,intergenic_variant,intron_variant,missense_variant,protein_altering_variant,splice_acceptor_variant,splice_donor_variant,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant,transcript_ablation
2,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,0.000078,0.040000,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
13,1,0.000050,0.054000,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
14,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120172,2,0.034357,0.064646,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
120173,1,0.037854,0.056886,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
120174,1,0.037854,0.056886,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
120175,1,0.037854,0.056886,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [69]:
res = df_IN[(df_IN['AF'] < 0.05) & (df_IN['predict'] > 0.8)]
res

,#CHROM,ID,REF,ALT,Consequence annotations from Ensembl VEP. Format: Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,REFSEQ_MATCH,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,AA_AF,EA_AF,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,ada_score,rf_score,BLOSUM62,CADD_phred,CADD_raw,CADD_raw_rankscore,DANN_rankscore,DANN_score,Eigen-PC-raw,Eigen-raw,FATHMM_pred,GERP++_NR,GERP++_RS,LRT_pred,M-CAP_pred,MetaLR_pred,MetaSVM_pred,MutPred_rankscore,MutPred_score,MutationAssessor_pred,MutationTaster_pred,PROVEAN_pred,Polyphen2_HDIV_pred,Polyphen2_HVAR_pred,REVEL_rankscore,REVEL_score,SIFT_pred,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,VEST3_rankscore,VEST3_score,phastCons100way_vertebrate,phastCons20way_mammalian,phyloP100way_vertebrate,phyloP20way_mammalian,ClinVar,ClinVar_ALLELEID,ClinVar_CLNDN,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,HGMD,HGMD_CLASS,HGMD_PHENO,HGMD_PUBMED,HGMD_RANKSCORE,DOMAINS_Pfam,SuperDups,gnomadWES,gnomadWES_AC_clean,gnomadWES_AF_clean,gnomadWES_AN_clean,gnomadWES_AC_Male,gnomadWES_AC_Female,gnomadWES_AN_Male,gnomadWES_AN_Female,gnomadWES_AF_Male,gnomadWES_AF_Female,gnomadWES_AC_raw,gnomadWES_AN_raw,gnomadWES_AF_raw,gnomadWES_Hom_AFR,gnomadWES_Hom_AMR,gnomadWES_Hom_ASJ,gnomadWES_Hom_EAS,gnomadWES_Hom_FIN,gnomadWES_Hom_NFE,gnomadWES_Hom_OTH,gnomadWES_Hom_clean,gnomadWES_Hom_raw,gnomadWES_AC_AFR,gnomadWES_AC_AMR,gnomadWES_AC_ASJ,gnomadWES_AC_EAS,gnomadWES_AC_FIN,gnomadWES_AC_NFE,gnomadWES_AC_OTH,gnomadWES_AN_AFR,gnomadWES_AN_AMR,gnomadWES_AN_ASJ,gnomadWES_AN_EAS,gnomadWES_AN_FIN,gnomadWES_AN_NFE,gnomadWES_AN_OTH,gnomadWES_AF_AFR,gnomadWES_AF_AMR,gnomadWES_AF_ASJ,gnomadWES_AF_EAS,gnomadWES_AF_SAS,gnomadWES_AF_FIN,gnomadWES_AF_NFE,gnomadWES_AF_OTH,gnomadWES_POPMAX,gnomadWES_AC_POPMAX,gnomadWES_AN_POPMAX,gnomadWES_AF_POPMAX,gnomadWES_Hom_Male,gnomadWES_Hom_Female,gnomadWGS,gnomadWGS_AC_clean,gnomadWGS_AF_clean,gnomadWGS_AN_clean,gnomadWGS_AC_Male,gnomadWGS_AC_Female,gnomadWGS_AN_Male,gnomadWGS_AN_Female,gnomadWGS_AF_Male,gnomadWGS_AF_Female,gnomadWGS_AC_raw,gnomadWGS_AN_raw,gnomadWGS_AF_raw,gnomadWGS_Hom_AFR,gnomadWGS_Hom_AMR,gnomadWGS_Hom_ASJ,gnomadWGS_Hom_EAS,gnomadWGS_Hom_FIN,gnomadWGS_Hom_NFE,gnomadWGS_Hom_OTH,gnomadWGS_Hom_clean,gnomadWGS_Hom_raw,gnomadWGS_AC_AFR,gnomadWGS_AC_AMR,gnomadWGS_AC_ASJ,gnomadWGS_AC_EAS,gnomadWGS_AC_FIN,gnomadWGS_AC_NFE,gnomadWGS_AC_OTH,gnomadWGS_AN_AFR,gnomadWGS_AN_AMR,gnomadWGS_AN_ASJ,gnomadWGS_AN_EAS,gnomadWGS_AN_FIN,gnomadWGS_AN_NFE,gnomadWGS_AN_OTH,gnomadWGS_AF_AFR,gnomadWGS_AF_AMR,gnomadWGS_AF_ASJ,gnomadWGS_AF_EAS,gnomadWGS_AF_FIN,gnomadWGS_AF_NFE,gnomadWGS_AF_OTH,gnomadWGS_POPMAX,gnomadWGS_AC_POPMAX,gnomadWGS_AN_POPMAX,gnomadWGS_AF_POPMAX,gnomadWGS_Hom_Male,gnomadWGS_Hom_Female,Cosmic,Cosmic_LEGACY_ID,Cosmic_CNT,Mastermind,Mastermind_MMURI3,3_prime_UTR_variant,5_prime_UTR_variant,coding_sequence_variant,downstream_gene_variant,frameshift_variant,incomplete_terminal_codon_variant,inframe_deletion,inframe_insertion,intergenic_variant,intron_variant,missense_variant,non_coding_transcript_exon_variant,non_coding_transcript_variant,protein_altering_variant,splice_acceptor_variant,splice_donor_variant,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant,predict
13150,chr2,rs202102082,T,G,G,splice_acceptor_variant,0,ANKRD36C,400986.0,Transcript,NM_001310154.1,protein_coding,NaN,20/88,NM_001310154.1:c.1532-2A>C,NaN,NaN,NaN,NaN,NaN,NaN,rs202102082,NaN,-1.0,NaN,SNV,EntrezGene,NaN,YES,NaN,NaN,NaN,NP_001297083.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [67]:
graph = xgb.to_graphviz(model_in, num_trees=0, **{'size': str(10)})
graph.render(filename='xgb.dot')

'xgb.dot.pdf'